In [1]:
import os
import sys


from azureml.core import Workspace
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException


from azureml.pipeline.core import Pipeline
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (numpy 1.21.5 (c:\users\xa155xb\anaconda3\envs\walmart2\lib\site-packages), Requirement.parse('numpy<=1.19.3; sys_platform == "win32"'), {'azureml-dataset-runtime'}).


### Create interface for AzureML Workspace

In [2]:
ws = Workspace.get(name="Walmart-Sales", 
    subscription_id =  "ef0073f1-56e3-462f-80b3-3beb320211e4",
    resource_group = "Walmart-1"
)

### Pipeline Arguments

In [3]:
# Arguments
# EXPERIMENT_NAME = PipelineParameter(
#     name="experiment_name",
#     default_value="data-processing-test-1"
# )

CONFIG = PipelineParameter(
    name="config",
    default_value="{'demand': {'group': ['Region'], 'time': 'Date', 'features': {'Weekly_Sales': ['impute_mdian']}}}"
)

### Pipeline Scripts

In [4]:
# Set the source directory of the python script to run
src_dir = os.path.abspath(path="../src")
src_dir

'c:\\Users\\XA155XB\\OneDrive - EY\\Documents\\GitHub\\WalmartSales\\src'

In [5]:
# Configuring the AzureML Run
aml_run_config = RunConfiguration()

# Set the run environment
env = Environment.get(
    workspace = ws,
    name = "walmart2"
)
aml_run_config.environment = env

# Set the compute target for the run
compute_target = ws.compute_targets["compute3"]
aml_run_config.target = compute_target

In [6]:
# Setting up the pipeline script step
process_data_step = PythonScriptStep(
    script_name="process_data.py",
    source_directory= src_dir,
    # arguments=["--experiment_name", EXPERIMENT_NAME, "--config", CONFIG],
    arguments=["--config", CONFIG],
    runconfig = aml_run_config,
    allow_reuse=False
)

### Creating and Publishing AzureML Pipeline

In [7]:
# Creating the pipeline with the python script steps
data_pipeline = Pipeline(workspace=ws, 
    steps=[process_data_step]   # More python script steps can be added to the list in the pipeline
)

In [8]:
# Checking the pipeline
data_pipeline.validate()

Step process_data.py is ready to be created [4484198e]


[]

In [9]:
# Publishing the pipeline
data_pipeline.publish(
    name = "Data_Pipeline",
    description = "Data pipeline to get data from different sources"
)

Created step process_data.py [4484198e][674649a9-87d6-40d6-b5c4-0683895ec733], (This step will run and generate new outputs)


Name,Id,Status,Endpoint
Data_Pipeline,37913fef-6672-4943-89d6-4a3283e8b052,Active,REST Endpoint


### Triggering the Pipeline

In [25]:
# from azureml.core.authentication import AzureCliAuthentication
from azureml.pipeline.core import Pipeline, PublishedPipeline
from azureml.core import Experiment

import requests

In [33]:
# Parameters
experiment_name = 'data-processing-test-1'
pipeline_id = "37913fef-6672-4943-89d6-4a3283e8b052"

In [34]:
ws = Workspace.get(name="Walmart-Sales", 
    subscription_id =  "ef0073f1-56e3-462f-80b3-3beb320211e4",
    resource_group = "Walmart-1",
)

In [38]:
published_pipeline = PublishedPipeline.get(ws, pipeline_id)
experiment = ws.experiments[experiment_name]

In [39]:
# Method 2
pipeline_run = experiment.submit(published_pipeline,
                                    continue_on_step_failure=True,
                                    pipeline_parameters={
                                        "config": "{'demand': {'group': ['Region'], 'time': 'Date', 'features': {'Weekly_Sales': ['impute_mdian']}}}"
                                        },
                                    # parent_run_id="<run_id>"
)

Submitted PipelineRun b56ef0fc-32bb-490f-9016-2e44258b697a


C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/data-processing-test-1/runs/b56ef0fc-32bb-490f-9016-2e44258b697a?wsid=/subscriptions/ef0073f1-56e3-462f-80b3-3beb320211e4/resourcegroups/Walmart-1/workspaces/Walmart-Sales


In [ ]:
# # Method 3
# cli_auth = AzureCliAuthentication()

# ws = Workspace.get(name="Walmart-Sales", 
#     subscription_id =  "ef0073f1-56e3-462f-80b3-3beb320211e4",
#     resource_group = "Walmart-1",
#     auth = cli_auth
# )